In [1]:
import pandas as pd
import re

def clean_name(product_name: str) -> str:
    product_name = re.sub(r'', '', product_name)

DATA = pd.read_csv('russian_supermarket_prices.csv')
DATA = DATA[DATA.apply(lambda row: str(row['package_size']) in row['product_name'], axis=1)]
DATA = DATA[DATA.apply(lambda row: str(row['unit']) in row['product_name'], axis=1)]
DATA = DATA[DATA['product_name'].str.contains('«')]
DATA

,product_name,product_category,brand,manufacturer,package_size,unit,old_price,new_price,catalogue_name,catalogue_date,year,source_url
0,"Молоко «Правильное» 3,2%-4%, 900 мл",Молоко,Правильное,NaN,900,мл,100.17,85.9,Сезонный каталог Атак «Недельный каталог»,с 17 по 23 марта 2022,2022,https://proshoper.ru//actions/atak/moskva/175686/
1,"Бифилайф «Рузский» кисломолочный2,5 %, 250 г","Кефир, ряженка, тан, айран",Рузское молоко,Рузское молоко,250,г,108.64,69.9,Сезонный каталог Атак «Недельный каталог»,с 17 по 23 марта 2022,2022,https://proshoper.ru//actions/atak/moskva/175686/
3,"Кефир «Домик в деревне» лёгкий вечер 1%, 900 г","Кефир, ряженка, тан, айран",Домик в деревне,Вимм-Билль-Данн,900,г,85.04,67.9,Сезонный каталог Атак «Недельный каталог»,с 17 по 23 марта 2022,2022,https://proshoper.ru//actions/atak/moskva/175686/
4,Сырок глазированный «Чудо Десерт» Три шоколада...,Сырок,Чудо,Вимм-Билль-Данн,40,г,41.78,31.9,Сезонный каталог Атак «Недельный каталог»,с 17 по 23 марта 2022,2022,https://proshoper.ru//actions/atak/moskva/175686/
5,Сырок глазированный «Чудо Десерт» Вишня и Пиро...,Сырок,Чудо,Вимм-Билль-Данн,40,г,41.78,31.9,Сезонный каталог Атак «Недельный каталог»,с 17 по 23 марта 2022,2022,https://proshoper.ru//actions/atak/moskva/175686/
...,...,...,...,...,...,...,...,...,...,...,...,...
44664,"Зефир «Полет», в глазури из темного шоколада, ...",Зефир,Полет,NaN,500,г,205.00,119.0,Каталог акций «Ашан»\n — Москва,с 11 по 20 ноября 2019,2019,https://proshoper.ru//actions/ashan/moskva/85675/
44671,Конфеты шоколадные «КОМИЛЬФО» в наборах крем м...,Конфеты,Комильфо,Nestlé,116,г,299.00,149.0,Каталог акций «Ашан»\n — Москва,с 11 по 20 ноября 2019,2019,https://proshoper.ru//actions/ashan/moskva/85675/
44672,Конфеты шоколадные «КОМИЛЬФО» в наборах фисташ...,Конфеты,Комильфо,Nestlé,116,г,299.00,149.0,Каталог акций «Ашан»\n — Москва,с 11 по 20 ноября 2019,2019,https://proshoper.ru//actions/ashan/moskva/85675/
44688,"Грибы «Лесная Былина» лисички, 440 г",Грибы,Лесная Былина,NaN,440,г,331.00,249.0,Каталог акций «Ашан»\n — Москва,с 11 по 20 ноября 2019,2019,https://proshoper.ru//actions/ashan/moskva/85675/


In [2]:
import re
import pandas as pd

def clean_text(text):
    return re.sub(r'(?<!\d),(?!\d)', '', text)

def bio_tag(tokens, data_row):
    tags = ["O"] * len(tokens)

    # --- BRAND ---
    text_original = clean_text(str(data_row.get("product_name", "")))

    # Найти все группы внутри кавычек
    brand_spans = re.findall(r"[«\"](.*?)[»\"]", text_original)
    brand_tokens = [span.split() for span in brand_spans if span.strip()]

    is_first = True
    for group in brand_tokens:
        for i in range(len(tokens) - len(group) + 1):
            window = tokens[i:i+len(group)]
            if window == group:
                for j in range(len(group)):
                    tags[i+j] = "B-BRAND" if is_first else "I-BRAND"
                    is_first = False

    # --- TYPE ---
    is_first = True
    if "B-BRAND" in tags:
        first_brand_idx = tags.index("B-BRAND")
        for i in range(0, first_brand_idx):
            if tags[i] == "O":
                tags[i] = "B-TYPE" if is_first else "I-TYPE"
                is_first = False

    # --- PERCENT ---
    is_first = True
    for i, tok in enumerate(tokens):
        if re.search(r"\d+[.,]?\d*%(-\d+[.,]?\d*%)?", tok):
            tags[i] = "B-PERCENT" if is_first else "I-PERCENT"

    # --- VOLUME ---
    size = str(data_row.get("package_size", "")).lower().split()[0]
    unit = str(data_row.get("unit", "")).lower()
    is_first = True
    for i, tok in enumerate(tokens):
        if tok.lower() == size or tok.lower() == unit:
            tags[i] = "B-VOLUME" if is_first else "I-VOLUME"
            is_first = False

    return tags

def build_span_dataset(DATA):
    dataset = []
    for _, row in DATA.iterrows():
        # Убираем кавычки из строки
        sample_text = clean_text(str(row["product_name"])).replace("«", "").replace("»", "").replace("\"", "")
        tokens = sample_text.split()
        tags = bio_tag(tokens, row)

        # start_index и end_index для каждого токена
        spans = []
        cursor = 0
        for tok, label in zip(tokens, tags):
            start_index = sample_text.find(tok, cursor)
            end_index = start_index + len(tok)
            spans.append((start_index, end_index, label))
            cursor = end_index + 1  # +1 чтобы пропустить пробел

        dataset.append({
            "sample": sample_text,
            "annotation": spans
        })
    return pd.DataFrame(dataset)


SPAN_DATA = build_span_dataset(DATA)
print(SPAN_DATA.head(5).to_string())

SPAN_DATA.to_csv('new_data.csv', sep=';', index=False)

                                                        sample                                                                                                                                                                   annotation
0                             Молоко Правильное 3,2%-4% 900 мл                                                                              [(0, 6, B-TYPE), (7, 17, B-BRAND), (18, 25, B-PERCENT), (26, 29, B-VOLUME), (30, 32, I-VOLUME)]
1                    Бифилайф Рузский кисломолочный2,5 % 250 г                                                                         [(0, 8, B-TYPE), (9, 16, B-BRAND), (17, 33, O), (34, 35, O), (36, 39, B-VOLUME), (40, 41, I-VOLUME)]
2                  Кефир Домик в деревне лёгкий вечер 1% 900 г              [(0, 5, B-TYPE), (6, 11, B-BRAND), (12, 13, I-BRAND), (14, 21, I-BRAND), (22, 28, O), (29, 34, O), (35, 37, B-PERCENT), (38, 41, B-VOLUME), (42, 43, I-VOLUME)]
3      Сырок глазированный Чудо Десерт Три шоколада 24,4